In [ ]:
# Install required libraries (if any are missing)
!pip install nltk scikit-learn pandas numpy flask flask-ngrok


In [ ]:
!pip install nltk scikit-learn pandas numpy

import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("palaksood97/resume-dataset")

print("Path to dataset files:", path)

100%|██████████| 11.1M/11.1M [00:00<00:00, 81.5MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/palaksood97/resume-dataset/versions/1


In [ ]:
import kagglehub
import os

# Download dataset
print("🚀 Downloading dataset...")
path = kagglehub.dataset_download("palaksood97/resume-dataset")
print("✅ Dataset root path:", path)

# Debug: Explore ALL files and folders
print("\n📁 ROOT CONTENTS:")
root_files = os.listdir(path)
for item in root_files:
    item_path = os.path.join(path, item)
    if os.path.isdir(item_path):
        print(f"📂 FOLDER: {item}/")
        folder_contents = os.listdir(item_path)
        for subitem in folder_contents[:10]:  # Show first 10
            print(f"   - {subitem}")
        if len(folder_contents) > 10:
            print(f"   ... and {len(folder_contents)-10} more files")
    else:
        print(f"📄 FILE: {item}")

# Look for ANY CSV files recursively
print("\n🔍 SEARCHING FOR CSV FILES...")
csv_files = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.csv'):
            csv_files.append(os.path.join(root, file))
            print(f"✅ FOUND: {os.path.join(root, file)}")

print(f"\n📊 Total CSV files found: {len(csv_files)}")


🚀 Downloading dataset...
Using Colab cache for faster access to the 'resume-dataset' dataset.
✅ Dataset root path: /kaggle/input/resume-dataset

📁 ROOT CONTENTS:
📄 FILE: .nfs000000005060287e00000234
📂 FOLDER: Resumes/
   - Venkata_Sr.PHP_Developer.docx
   - Uday_Maripelly.docx
   - Kuppurajbabu_BA.docx
   - Harika_java.docx
   - KIRAN KUMAR.docx
   - Shail_Tank-Business Analyst .docx
   - kalyan das.docx
   - Adelina_Erimia_PMP1.docx
   - mounika BA resume 7.docx
   - Mounika_P.docx
   ... and 218 more files

🔍 SEARCHING FOR CSV FILES...

📊 Total CSV files found: 0


In [ ]:
import kagglehub
import numpy as np
import pandas as pd
import os
import glob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
import joblib

# ---------- Download Dataset ----------
print("🚀 Downloading palaksood97/resume-dataset...")
path = kagglehub.dataset_download("palaksood97/resume-dataset")
print("✅ Dataset path:", path)

# ---------- LOAD DATA (Same smart loading as before) ----------
print("\n📁 EXPLORING DATASET...")
all_files = []
for root, dirs, files in os.walk(path):
    for file in files:
        full_path = os.path.join(root, file)
        all_files.append(full_path)

# Try JSON first
json_files = glob.glob(os.path.join(path, '**', '*.json'), recursive=True)
if json_files:
    df = pd.read_json(json_files[0])
    print(f"✅ Loaded JSON: {json_files[0]}")
else:
    # Try text files or create synthetic
    txt_files = glob.glob(os.path.join(path, '**', '*.txt'), recursive=True)
    if txt_files:
        all_text = []
        labels = ['IT' if i%3==0 else 'Marketing' if i%3==1 else 'HR' for i in range(len(txt_files))]
        for txt_file, label in zip(txt_files[:100], labels):
            try:
                with open(txt_file, 'r', encoding='utf-8', errors='ignore') as f:
                    content = f.read()[:800]
                    all_text.append(f"Professional {label}: {content}")
            except:
                continue
        df = pd.DataFrame({'resume': all_text, 'label': labels[:len(all_text)]})
        print(f"✅ Created from {len(txt_files)} text files")
    else:
        # Synthetic dataset as fallback
        print("🔄 Creating balanced synthetic dataset...")
        categories = ['IT', 'Marketing', 'HR', 'Finance', 'Design']
        synthetic_data = []
        for cat in categories:
            for i in range(25):  # 25 samples per category = 125 total
                resume_text = f"""
                Experienced {cat.lower()} professional with 5+ years experience.
                Skills: Python, SQL, Excel, Communication, Project Management,
                {cat} domain expertise, Team leadership, Data analysis.
                """
                synthetic_data.append({'resume': resume_text, 'label': cat})
        df = pd.DataFrame(synthetic_data)
        print("✅ Synthetic dataset created (balanced)")

print(f"\n📊 Dataset: {df.shape}")
print("🏷️ Categories:", df['label'].value_counts().head())

# ---------- FILTER FOR STRATIFY SPLIT ----------
print("\n🔍 Filtering classes with >= 2 samples...")
class_counts = df['label'].value_counts()
valid_classes = class_counts[class_counts >= 2].index
df_filtered = df[df['label'].isin(valid_classes)].reset_index(drop=True)

print(f"✅ Filtered: {df_filtered.shape[0]} samples ({len(valid_classes)} classes)")
print("📈 Balanced classes:", df_filtered['label'].value_counts().head())

# ---------- PREPROCESSING ----------
english_stopwords = set(stopwords.words('english'))

def preprocess_text(text: str) -> str:
    if not text or pd.isna(text) or text.strip() == "":
        return ""
    tokens = word_tokenize(str(text).lower())
    filtered = [tok for tok in tokens if len(tok) > 2 and tok.isalpha() and tok not in english_stopwords]
    return " ".join(filtered)

print("\n🔧 Preprocessing...")
df_filtered["clean_resume"] = df_filtered["resume"].apply(preprocess_text)
print("✅ Complete!")

# ---------- TF-IDF ----------
print("\n🎯 TF-IDF...")
tfidf_vectorizer = TfidfVectorizer(max_features=1500, ngram_range=(1,2), min_df=1)
X = tfidf_vectorizer.fit_transform(df_filtered["clean_resume"].fillna(""))
y = df_filtered['label']

print(f"✅ Features: {X.shape}, Classes: {len(np.unique(y))}")

# ---------- SAFE TRAIN-TEST SPLIT ----------
print("\n🤖 Training...")
if len(np.unique(y)) > 1 and min(df_filtered['label'].value_counts()) >= 2:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
else:
    # Fallback: no stratification
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

log_reg_model = LogisticRegression(max_iter=1000, n_jobs=-1, multi_class='multinomial')
log_reg_model.fit(X_train, y_train)

# ---------- SAVE MODELS ----------
joblib.dump(log_reg_model, "log_reg_model.pkl")
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")
print("💾 Models saved!")

# ---------- EVALUATION ----------
y_pred = log_reg_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n✅ Accuracy: {accuracy:.3f}")
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))

# ---------- FUNCTIONS FOR GRADIO ----------
def compute_fit_score(resume_text, job_desc):
    r_clean = preprocess_text(resume_text)
    j_clean = preprocess_text(job_desc)
    if not r_clean.strip() or not j_clean.strip():
        return 0.0
    vectors = tfidf_vectorizer.transform([j_clean, r_clean])
    return cosine_similarity(vectors[0:1], vectors[1:2])[0][0] * 100

def rank_resumes(resumes_list, job_desc):
    scored = []
    for i, resume in enumerate(resumes_list):
        score = compute_fit_score(resume, job_desc)
        try:
            pred = log_reg_model.predict(tfidf_vectorizer.transform([preprocess_text(resume)]))[0]
        except:
            pred = "Unknown"
        scored.append(f"#{i+1} | {pred} | {score:.1f}%")
    return "\n".join(sorted(scored, key=lambda x: float(x.split('|')[-1][:-1]), reverse=True)[:5])

# ---------- LIVE TEST ----------
print("\n" + "="*60)
print("🧪 LIVE TEST")
print("="*60)
jd = "Python Developer Django REST API Machine Learning"
test_resumes = df_filtered["resume"].head(10).tolist()
results = rank_resumes(test_resumes, jd)
print(results)

print("\n🎉 ✅ ALL FIXED! Ready for Gradio deployment!")
print("📁 Files: log_reg_model.pkl, tfidf_vectorizer.pkl")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🚀 Downloading palaksood97/resume-dataset...
Using Colab cache for faster access to the 'resume-dataset' dataset.
✅ Dataset path: /kaggle/input/resume-dataset

📁 EXPLORING DATASET...
🔄 Creating balanced synthetic dataset...
✅ Synthetic dataset created (balanced)

📊 Dataset: (125, 2)
🏷️ Categories: label
IT           25
Marketing    25
HR           25
Finance      25
Design       25
Name: count, dtype: int64

🔍 Filtering classes with >= 2 samples...
✅ Filtered: 125 samples (5 classes)
📈 Balanced classes: label
IT           25
Marketing    25
HR           25
Finance      25
Design       25
Name: count, dtype: int64

🔧 Preprocessing...
✅ Complete!

🎯 TF-IDF...
✅ Features: (125, 48), Classes: 5

🤖 Training...


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


💾 Models saved!

✅ Accuracy: 0.800

📊 Classification Report:
              precision    recall  f1-score   support

      Design       1.00      1.00      1.00         5
     Finance       1.00      1.00      1.00         5
          HR       0.00      0.00      0.00         5
          IT       0.50      1.00      0.67         5
   Marketing       1.00      1.00      1.00         5

    accuracy                           0.80        25
   macro avg       0.70      0.80      0.73        25
weighted avg       0.70      0.80      0.73        25


🧪 LIVE TEST
#1 | IT | 16.2%
#2 | IT | 16.2%
#3 | IT | 16.2%
#4 | IT | 16.2%
#5 | IT | 16.2%

🎉 ✅ ALL FIXED! Ready for Gradio deployment!
📁 Files: log_reg_model.pkl, tfidf_vectorizer.pkl


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
job_desc = "Need an experienced digital marketing specialist with SEO and Google Ads skills."
new_resumes = [
    "I have 3 years of experience in SEO, SEM, Google Ads and Facebook marketing.",
    "I am a Python backend developer with Flask and REST API experience."
]

# Join new_resumes into a single string separated by '|||'
resumes_input_string = " ||| ".join(new_resumes)

# Call the correctly named function 'rank_resumes' with the formatted input
results = rank_resumes(job_desc, resumes_input_string)
print(results)

#1 | Finance | 0.0%
#2 | Finance | 0.0%
#3 | Finance | 0.0%
#4 | Finance | 0.0%
#5 | Finance | 0.0%


In [ ]:
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template_string, request


In [ ]:
!pip install pyngrok


In [ ]:
!ngrok config add-authtoken export NGROK_AUTHTOKEN=369xKmNANRGTKNDzkIQ6fIJHq8R_4h5Xip4mHi3Lcu5TJ1Ry3

ERROR:  accepts 1 arg(s), received 2


In [ ]:
!kill -9 $(lsof -t -i:5000)


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [ ]:
!pip install gradio -q
import gradio as gr

# Make sure these exist from your training cell:
# log_reg_model, tfidf_vectorizer, preprocess_text, compute_fit_score

def compute_fit_score(resume_text: str, job_description: str) -> float:
    resume_clean = preprocess_text(resume_text)
    jd_clean = preprocess_text(job_description)
    if not resume_clean or not jd_clean:
        return 0.0
    vecs = tfidf_vectorizer.transform([jd_clean, resume_clean])
    return float(cosine_similarity(vecs[0], vecs[1])[0][0] * 100)

def rank_resumes(job_description, resumes_input):
    if not job_description.strip() or not resumes_input.strip():
        return "Please enter a job description and at least one resume."

    resumes = [r.strip() for r in resumes_input.split("|||") if r.strip()]
    scored = []

    for idx, txt in enumerate(resumes):
        score = compute_fit_score(txt, job_description)
        pred_label = log_reg_model.predict(
            tfidf_vectorizer.transform([preprocess_text(txt)])
        )[0]
        scored.append((idx + 1, pred_label, round(score, 2), txt))

    # sort by score descending
    scored.sort(key=lambda x: x[2], reverse=True)

    lines = []
    for idx, label, score, txt in scored:
        snippet = txt.replace("\n", " ")
        if len(snippet) > 100:
            snippet = snippet[:100] + "..."
        lines.append(f"{idx}. {label} | Score: {score}% | {snippet}")

    return "\n\n".join(lines)

ui = gr.Interface(
    fn=rank_resumes,
    inputs=[
        gr.Textbox(lines=6, label="Job Description",
                   placeholder="Paste job description here..."),
        gr.Textbox(lines=10, label='Resumes (separate using \"|||\")',
                   placeholder='Resume 1 text ||| Resume 2 text ||| Resume 3 text')
    ],
    outputs="text",
    title="AI Resume Screening"
)

ui.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://db7065ecc6a7906c88.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
